ROC ---. The x-axis indicates the False Positive Rate and the y-axis indicates the True Positive Rate.
ROC Curve: Plot of False Positive Rate (x) vs. True Positive Rate (y).
    .The area under the curve can be calculated to give a single score for a classifier model across all threshold values. 
    This is called the ROC area under curve or ROC AUC or sometimes ROCAUC.
    .The threshold is applied to the cut-off point in probability between the positive and negative classes,
    which by default for any classifier would be set at 0.5, halfway between each outcome (0 and 1).

In [7]:
import pandas as pd
df=pd.read_csv('creditcard.csv')



In [8]:
df.shape


(284807, 31)

In [9]:
df.isnull().count()

Time      284807
V1        284807
V2        284807
V3        284807
V4        284807
V5        284807
V6        284807
V7        284807
V8        284807
V9        284807
V10       284807
V11       284807
V12       284807
V13       284807
V14       284807
V15       284807
V16       284807
V17       284807
V18       284807
V19       284807
V20       284807
V21       284807
V22       284807
V23       284807
V24       284807
V25       284807
V26       284807
V27       284807
V28       284807
Amount    284807
Class     284807
dtype: int64

In [10]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [11]:
X=df.drop('Class',axis=1)
y=df.Class

### Crossvalidation like KFold and Hyperparameter tuning technique for handling imbalanced data

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV

In [13]:
log_class=LogisticRegression()


### see hyperparameters for LogistivRegression below

###C : float, default=1.0
    Inverse of regularization strength; must be a positive float.
    Like in support vector machines, smaller values specify stronger
    regularization.

###penalty : {'l1', 'l2', 'elasticnet', 'none'}, default='l2'
    Used to specify the norm used in the penalization. The 'newton-cg',
    'sag' and 'lbfgs' solvers support only l2 penalties. 'elasticnet' is
    only supported by the 'saga' solver. If 'none' (not supported by the
    liblinear solver), no regularization is applied.

In [14]:
grid={'C':10.0 **np.arange(-2,3), 'penalty':['l1','l2']}
cv=KFold(n_splits=5,shuffle=False, random_state=None)

In [28]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7)

In [29]:
clf=GridSearchCV(log_class,grid,cv=cv,n_jobs=-1,scoring='f1_macro')
clf.fit(X_train,y_train)

C:\Users\mkaze\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.8230882         nan 0.8292115         nan 0.82335649
        nan 0.84246935        nan 0.82807447]
  warnings.warn(
C:\Users\mkaze\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

In [ ]:
### we can use accuracy instead of f1_macro ( is an scoring parameter)

###Cross-validation is a technique for evaluating ML models by training several ML models on subsets of 
the available input data and evaluating them on the complementary subset of the data. ... In k-fold cross-validation,
you split the input data into k subsets of data (also known as folds).

In [43]:
y_pred=clf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85273    49]
 [   46    75]]
0.9988881476539916
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85322
           1       0.60      0.62      0.61       121

    accuracy                           1.00     85443
   macro avg       0.80      0.81      0.81     85443
weighted avg       1.00      1.00      1.00     85443



### now we try an ensemble model like RandomForest to see the difference

In [17]:
X_train.shape

(199364, 30)

In [31]:
y_train.value_counts()

0    199016
1       348
Name: Class, dtype: int64

In [48]:
### we give a weight to RandomForest classifier , keep weight of 0s and increase 1s , 100%

In [50]:
class_weight=dict({0:1,1:100})

In [ ]:
###without calss_weight--- see result Ln 45


###from sklearn.ensemble import RandomForestClassifier
###classifier =RandomForestClassifier()
###classifier.fit(X_train,y_train)

In [51]:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(class_weight=class_weight)
classifier.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 1, 1: 100})

In [52]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85320     2]
 [   28    93]]
0.9996488887328394
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85322
           1       0.98      0.77      0.86       121

    accuracy                           1.00     85443
   macro avg       0.99      0.88      0.93     85443
weighted avg       1.00      1.00      1.00     85443



In [53]:
### results below are without class_weight

In [45]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85318     4]
 [   29    92]]
0.9996137776061234
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85322
           1       0.96      0.76      0.85       121

    accuracy                           1.00     85443
   macro avg       0.98      0.88      0.92     85443
weighted avg       1.00      1.00      1.00     85443



###Under Sampling technique

In [19]:
y_train.value_counts()

0    199029
1       335
Name: Class, dtype: int64

In [25]:
from imblearn import under_sampling

In [36]:
from collections import Counter
from imblearn.under_sampling import NearMiss
ns=NearMiss(0.8)
X_train_ns,y_train_ns=ns.fit_resample(X_train,y_train)
print('The number of classes before fit{}'.format(Counter(y_train)))
print('The number of classes after fit{}'.format(Counter(y_train_ns)))

The number of classes before fitCounter({0: 199016, 1: 348})
The number of classes after fitCounter({0: 435, 1: 348})


In [37]:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [38]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[67871 17428]
 [    9   135]]
0.7959224278173753
              precision    recall  f1-score   support

           0       1.00      0.80      0.89     85299
           1       0.01      0.94      0.02       144

    accuracy                           0.80     85443
   macro avg       0.50      0.87      0.45     85443
weighted avg       1.00      0.80      0.88     85443



### Over Sampling

In [40]:
from imblearn.over_sampling import RandomOverSampler

In [50]:
os=RandomOverSampler(0.5)
X_train_ns,y_train_ns=os.fit_resample(X_train,y_train)
print('The number of classes before fit{}'.format(Counter(y_train)))
print('The number of classes after fit{}'.format(Counter(y_train_ns)))

C:\Users\mkaze\anaconda3\lib\site-packages\imblearn\utils\_validation.py:587: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


The number of classes before fitCounter({0: 199016, 1: 348})
The number of classes after fitCounter({0: 199016, 1: 99508})


In [44]:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [45]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[85288    11]
 [   24   120]]
0.9995903701883126
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85299
           1       0.92      0.83      0.87       144

    accuracy                           1.00     85443
   macro avg       0.96      0.92      0.94     85443
weighted avg       1.00      1.00      1.00     85443



### SMOTETomek

In [55]:
from imblearn import combine

In [56]:
from imblearn.combine import SMOTETomek

In [ ]:
os=SMOTETomek(0.5)
X_train_ns,y_train_ns=os.fit_resample(X_train,y_train)
print('The number of classes before fit{}'.format(Counter(y_train)))
print('The number of classes after fit{}'.format(Counter(y_train_ns)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

In [ ]:
y_pred=classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

### ensemble techniques 

In [ ]:
### Easy Ensemble Classifier

In [ ]:
from imblearn import EasyEnsembleClassifier

In [ ]:
easy=EasyEnsembleClassifier()


In [ ]:
### Isolation Forest i sanother technique